<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>

## Introduction

<i>In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in Toronto. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.In order to get the data from Toronto Neighborhoods you will use Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 

## <strong><u> 1st Part:Data Scrapping

<i>Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None) #If set to None, the number of items to be printed is unlimited
pd.set_option('display.max_rows', None)    ##If set to None, the number of items to be printed is unlimited

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge geocoder --yes  #for getting data from the internet

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

<a id='item1'></a>

<i>Next, let's load the data.

In [2]:
import requests  #The requests library is the de facto standard for making HTTP requests in Python.

#Lets examine it further in our example to see if indeed the data are in json format
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)  
print(response)  #For example, a 200 OK status means that your request was successful, whereas a 404 NOT FOUND status means that the resource you were looking for was not found
print(response.status_code)
if 'json' in response.headers.get('Content-Type'):
    js = response.json()
else:
    print('Response content is not in JSON format.')
    js = 'spam'

<Response [200]>
200
Response content is not in JSON format.


<i>Usually if you take a look at the repsonse you can undestand if it is a json file or not.Here it is not.Thus you cannot use the json.load method or json() method and then use the json normalize method to convert them into a dataframe.As you observe the data in the wikipedia page are not in json format.Thus we cannot use the json normalize method to get the data into a pandas dataframe. Therefore we will have to read a html file with a different way.That is why we use BeatifulSoup Library

### Beautiful Soup Scrapping Method

<i>This library is used to parse.Beautiful Soup supports the HTML parser included in Python’s standard library, but it also supports a number of third-party Python parsers. One is the lxml parser. Depending on your setup, you might install lxml using:pip install lxml.
Parsing resolves (a sentence) into its component parts and describe their syntactic roles.

In [3]:
!pip install beautifulsoup4 #Used to scrape information from websites

In [4]:
!pip install lxml 

In [6]:
import requests
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#if you did not use text then you wouldn't have the result.you would have the 200 results which shows that the connection was successful.Witout text you will have:<Response [200]>

In [7]:
from bs4 import BeautifulSoup
import requests
soup=BeautifulSoup(response,'lxml')
#print(soup.prettify())

In [8]:
print(soup.title.text)

List of postal codes of Canada: M - Wikipedia


## Finding the headers of our dataframe

In [9]:
table = soup.find('table',class_='wikitable sortable')
#print(table)

In [10]:
print(type(table.tr.text))
table.tr.text #gives the first tr which is the headres of the dataframe that we want to create

<class 'str'>


'\nPostcode\nBorough\nNeighbourhood\n'

In [11]:
headers=table.tr.text.split("\n")
headers

['', 'Postcode', 'Borough', 'Neighbourhood', '']

In [12]:
del headers[0]
del headers[-1]
headers

['Postcode', 'Borough', 'Neighbourhood']

## Finding the contents of our dataframe

In [13]:
table = soup.find('table',class_='wikitable sortable')
#print(table)
#print(table.find_all('tr'))  #gives you a list.

In [14]:
print(table.find_all('tr')[0])  #select the first element of the list 
print(table.find_all('tr')[0].text)  #making the element a text
print(table.find_all('tr')[1]) 
print(table.find_all('tr')[1].text)

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>

Postcode
Borough
Neighbourhood

<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>

M1A
Not assigned
Not assigned



## 1st method  of scrapping data and creating a dataframe ,using csv file.

In [15]:
table1=""
for tr in table.find_all('tr'):
    row=""
    for td in tr.find_all('td'): #for every td in each tr you add it in the string and you create the first row
        row=row+","+td.text
    table1=table1+row[1:]  #you slice the string row from the first element in order to ignore the frist comma
#print(table1)
#You can see the strings ,each one representing a row

import csv
csv_file=open('toronto.csv','wb') #create a csv file and write in bytes.
#When writing in binary mode, Python makes no changes to data as it is written to the file. In text mode (when the b is excluded as in just w or when you specify text mode with wt),
#however, Python will encode the text based on the default text encoding. Additionally, Python will convert line endings (\n) to whatever the platform-specific line ending is, which would corrupt a binary file like an exe or png file.

csv_writer=csv.writer(csv_file)  #csv writer method of that csv module
#csv_writer.writerow(headers)  
csv_file.write(bytes(table1,encoding="ascii",errors="ignore")) #ascii:American Standard Code for Information Interchange
csv_file.close()
import pandas as pd
df=pd.read_csv('toronto.csv',names=headers)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 2nd method of scrapping data and creating a dataframe without using csv files.

In [16]:
#2nd way of crateing the dataframe without having to use csv files
borough=[]
postcode=[]
neighbourhood=[]
for tr in table.find_all('tr'):
    for i,tds in enumerate(tr.find_all('td')): 
        if i==0:
            postcode.append(tds.text)
        if i==1:
            borough.append(tds.text)
        if i==2:
            neighbourhood.append(tds.text)
print(borough) 
print("\n")
print(postcode)
print("\n")
print(neighbourhood) #As you observe it has \n in the end which should be removed:

neighbourhood=[value.strip() for value in neighbourhood]  #or map(str.strip,neighbourhood)

import pandas as pd
data={"Postcode":postcode,"Borough":borough,"Neighbourhood":neighbourhood}
df2=pd.DataFrame(data)
df2.head()

['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto', 'North York', 'North York', "Queen's Park", 'Not assigned', "Queen's Park", 'Scarborough', 'Scarborough', 'Not assigned', 'North York', 'East York', 'East York', 'Downtown Toronto', 'Downtown Toronto', 'North York', 'Not assigned', 'Not assigned', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Scarborough', 'Scarborough', 'Scarborough', 'Not assigned', 'North York', 'North York', 'East York', 'Downtown Toronto', 'York', 'Not assigned', 'Not assigned', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Scarborough', 'Scarborough', 'Scarborough', 'Not assigned', 'Not assigned', 'East Toronto', 'Downtown Toronto', 'York', 'Not assigned', 'Not assigned', 'Not assigned', 'Scarborough', 'Not assigned', 'Not assigned', 'East York', 'Downtown Toronto', 'Downtown Toronto', 'Not assigned', 'Not assigned', 'Not assigned', 'Scarborough', 'North York', 'North York', 'North York', 'North York', 'Eas

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 3rd method of scrapping data and creating a dataframe without using csv files.

In [17]:
#The above should be made into a list of lists
import pandas as pd
row=[]
for tr in table.find_all('tr'):
    data = (tr.find_all('td'))
    row.append([i.text.strip() for i in data])

df3=pd.DataFrame(data=row)
df3=df3.iloc[1:]
df3.columns=headers
df3.reset_index(drop=True,inplace=True)
df3.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 4th method of scrapping data and creating a dataframe without using csv files.

In [18]:
#The above should be made into a list of lists
import pandas as pd
row=[]
for tr in table.find_all('tr'):
    data = (tr.find_all('td'))
    row.append([i.text for i in data])

df4=pd.DataFrame(data=row)
df4=df4.iloc[1:]
df4.columns=headers
df4.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n


In [19]:
for i,column in enumerate(df4['Neighbourhood']):  #wrong way of iterration for a column
    #print(column)
    #df4['Neighbourhood']=column.strip() # pass the value each time in all the rows
    df4.loc[i+1,"Neighbourhood"]=column.strip() 
df4.reset_index(drop=True,inplace=True)  #reseting the index starts from 0
df4.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Now that we have obtained our data from Wikipedia let's start with the project.Lets use df from the 1st method!

In [20]:
table1=""
for tr in table.find_all('tr'):
    row=""
    for td in tr.find_all('td'): #for every td in each tr you add it in the string and you create the first row
        row=row+","+td.text
    table1=table1+row[1:]  #you slice the string row from the first element in order to ignore the frist comma
#print(table1)
#You can see the strings ,each one representing a row

import csv
csv_file=open('toronto.csv','wb') #create a csv file and write in bytes.
#When writing in binary mode, Python makes no changes to data as it is written to the file. In text mode (when the b is excluded as in just w or when you specify text mode with wt),
#however, Python will encode the text based on the default text encoding. Additionally, Python will convert line endings (\n) to whatever the platform-specific line ending is, which would corrupt a binary file like an exe or png file.

csv_writer=csv.writer(csv_file)  #csv writer method of that csv module
#csv_writer.writerow(headers)  
csv_file.write(bytes(table1,encoding="ascii",errors="ignore")) #ascii:American Standard Code for Information Interchange
csv_file.close()
import pandas as pd
df=pd.read_csv('toronto.csv',names=headers)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [21]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<i>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.Thus we drop the rows that have borough =Not assigned

In [22]:
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


<i>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [23]:
#In order to implement this we will use the groupby function and we will use either apply or agg function on the groupby object.
#DataFrameGroupBy.agg(arg, *args, **kwargs)[source]--->Aggregate using one or more operations over the specified axis.
df1=df.groupby(['Postcode']).count()
df1.head()
#As you see ,you can groupby only by postcode since each postcode is unique.However if you want to be completely safe you can groupby using Postcode and Borough as well:
#see below:

,Borough,Neighbourhood
Postcode,,
M1B,2,2
M1C,3,3
M1E,3,3
M1G,1,1
M1H,1,1


In [24]:
df1=df.groupby(['Postcode','Borough']).count()
df1.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,2
M1C,Scarborough,3
M1E,Scarborough,3
M1G,Scarborough,1
M1H,Scarborough,1


In [25]:
df1=df.groupby(['Postcode','Borough'])["Neighbourhood"].apply(",".join)
df1.head()

Postcode  Borough    
M1B       Scarborough                           Rouge,Malvern
M1C       Scarborough    Highland Creek,Rouge Hill,Port Union
M1E       Scarborough         Guildwood,Morningside,West Hill
M1G       Scarborough                                  Woburn
M1H       Scarborough                               Cedarbrae
Name: Neighbourhood, dtype: object

In [26]:
type(df1)

pandas.core.series.Series

In [27]:
df1.reset_index().head()  #reset_index:except from changing the numbering in the rows 
#reset_index: Generates a new DataFrame or Series with the index reset.

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [28]:
#However lets use the agg function which returns a dataframe and not a pandas.series as the apply function does.It is simplier and better
df= df.groupby(['Postcode','Borough'],sort=False).agg( ', '.join)
df.head()

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Not assigned


In [29]:
df.reset_index(inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


<i>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [30]:
df.loc[df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood
4,M7A,Queen's Park,Not assigned


In [31]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood']

4    Not assigned
Name: Neighbourhood, dtype: object

In [32]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
#or df['Neighbourhood'].replace('Not assigned',df['Borough'],inplace=True)

In [33]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [34]:
type(df)

pandas.core.frame.DataFrame

In [35]:
df.shape

(103, 3)

## <strong><u> 2nd Part:Get latitude and longitude coordinates of each neighborhood

<i> Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.We will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates.

In [36]:
#Use geolocator to find the latitudes and longitudes in Toronto
import geopy
address="Toronto ,Ontario"
geolocator = Nominatim(user_agent="toronto_explorer")
lat_lng_coords = None

while(lat_lng_coords is None):
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    lat_lng_coords=[latitude,longitude]

print('The geograpical coordinate of Toronto are {}, {}.'.format(lat_lng_coords[0], lat_lng_coords[1]))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<i> Now that we got the coordinates of Toronto let's get the coordinates for each postcode.However here we cannot use the same method as the one we used above since Nominatin can only be used for adresses and not for postcodes.We cannot also use Google API since it is not free anymore and it returns None.I found on internet another great API ,arcgis API.Thus this is what we will use to find the coordinates.

In [ ]:
#!conda install -c conda-forge geocoder --yes   #This has been implemented above
import geocoder
Latitude_list=[]
Longitude_list=[]
for i in range(df.shape[0]):
    address='{}, Toronto, Ontario'.format(df.at[i,'Postcode'])   #DataFrame.at--->Access a single value for a row/column label pair.
    g = geocoder.arcgis(address)
    Latitude_list.append(g.latlng[0])
    Longitude_list.append(g.latlng[1])

In [ ]:
df['Latitude']=Latitude_list
df['Longitude']=Longitude_list

In [ ]:
df.head()

#### Create a map of Toronto with neighborhoods superimposed on top.


In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng,postcode, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Postcode'],df['Borough'], df['Neighbourhood']):
    label = '{},{}, {}'.format(postcode,neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<i>Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.I have censored my credentials.

#### Define Foursquare Credentials and Version

In [40]:
# The code was removed by Watson Studio for sharing.

#### Let's create a function to explore the venues to all the neighborhoods in Toronto

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):  #Here you wont use apply  since it makes an itteration by itself in the df columns-series, inside the defined function.Where 
    #names=df['Neighborhood'],latitudes=df['Latitude'],longitudes=df['Longitude']
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):  
        #Here you wont use apply  since it makes an itteration by itself in the manhattam dataframe columns-series.We cant use also apply since apply is usually used in one column while here we 
        #heve more than one columns ,We have names,latitude and longitudes
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']   ##This outputs a list for each neighbourhood and has the vanues in each neighbourhood
        
        # return only relevant information for each nearby venue
        venues_list.append([(          #Imports every list to the venues_list
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results]) #here we did not use the get_category_type(row) user defined function since we know that the categories have data in the venues.categories
            #print(venues_list) #### If you want to understand how it passes the data in the list.
            #However you should know that this will take a lot of calculation time.
            #The loop for v in results applies for all the elements in the append,name ,
            #lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'] and v['venue']['categories'][0]['name']
    
    
    #The venues list will be something like that.I copied paste it from New York example from IBM since I do not want to print it for Toronto ,since it will take a lot of calculation time:
    #[[('Marble Hill', 40.87655077879964, -73.91065965862981, "Arturo's", 40.87441177110231, -73.91027100981574, 'Pizza Place'), 
    #  ('Marble Hill', 40.87655077879964, -73.91065965862981, 'Bikram Yoga', 40.876843690797934, -73.90620384419528, 'Yoga Studio'),........]]
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues) #returns a dataframe which we will later assign it to a value

In [42]:
#If this creates an error then run it again!This has to do with the Foursquare API.
#let's apply this function to our toronto dataframe:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights, Lawrence Manor
Queen's Park
Queen's Park
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The Danforth West, Riv

In [43]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75242,-79.329242,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75242,-79.329242,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.75242,-79.329242,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.73060,-79.313265,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.73060,-79.313265,Memories of Africa,43.726602,-79.312427,Grocery Store


In [44]:
toronto_venues.shape

(2470, 7)

## <strong><u> 3rd Part:Examine our data

<i>Let's check how many venues were returned for each neighborhood

In [45]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,14,14,14,14,14,14
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",1,1,1,1,1,1
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",14,14,14,14,14,14
"Alderwood, Long Branch",5,5,5,5,5,5
"Bathurst Manor, Downsview North, Wilson Heights",2,2,2,2,2,2
Bayview Village,5,5,5,5,5,5
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,63,63,63,63,63,63


#### Let's find out how many unique categories can be curated from all the returned venues

In [46]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 262 uniques categories.


### Analyze Each Neighborhood

In [47]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="") #Returns new dataframe
toronto_onehot.head()

,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [48]:
# add neighborhood column back to dataframe of dummies
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
toronto_onehot.head()

,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [49]:
#Lets create the columns that we want:
fixed_columns= ["Neighbourhood"]+ list(toronto_onehot.columns[:-1])  #adding two lists
#Now lets see the 5 first columns in the fixed columns list
fixed_columns[0:5]

['Neighbourhood',
 'Afghan Restaurant',
 'Airport',
 'American Restaurant',
 'Antique Shop']

In [50]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [51]:
toronto_onehot.shape

(2470, 263)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [52]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.03,0.0,0.01,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.030000,0.0,0.03,0.0,0.0,0.0,0.01,0.000000,0.0,0.0,0.0,0.0,0.02,0.0,0.01,0.02,0.0,0.0,0.000000,0.01,0.03

#### Let's confirm the new size

In [53]:
toronto_grouped.shape

(99, 263)

#### Let's print each neighborhood along with the top 5 most common venues

In [54]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})  #DataFrame.round(self, decimals=0, *args, **kwargs)[source].Round a DataFrame to a variable number of decimal places.
    #if you had many columns then you could use: temp.round(2) without having to specify the column
    #With a dict, the number of places for specific columns can be specified with the column names as key and the number of decimal places as value
    #e.g. df.round({'dogs': 1, 'cats': 0})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))  #gives you a dataframe
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.06
2        Hotel  0.05
3   Steakhouse  0.04
4          Bar  0.03


----Agincourt----
                venue  freq
0       Shopping Mall  0.14
1  Chinese Restaurant  0.14
2         Supermarket  0.14
3              Bakery  0.07
4                Pool  0.07


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                   Pharmacy   1.0
1          Afghan Restaurant   0.0
2               Noodle House   0.0
3         Mexican Restaurant   0.0
4  Middle Eastern Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.14
1  Japanese Restaurant  0.07
2           Beer Store  0.07
3          Coffee Shop  0.07
4         Liquor Store  0.07


----Alderwood, Long Branch----
               venue  freq
0  Convenience Store  

#### Let's put that into a *pandas* dataframe

<i>First, let's write a function to sort the venues in descending order.

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<i> Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd'] #Will be used for the columns in the new dataframe

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        #it goes here after the 3 first data since after that you have th is each nymber e.g 4th,5th etc.
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_neigh_venues_sorted = pd.DataFrame(columns=columns)
toronto_neigh_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']  #passed the data in our dataframe for the first column Nieghborhood


#Lets pass the data in the dataframe
for ind in np.arange(toronto_grouped.shape[0]):
    toronto_neigh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)  #ind here is 0 so it return the first row which has an index of 0.
    #neighborhoods_venues_sorted.iloc[ind, 1:] means ind row which starts from 0 and columns from 1 to the end ,since the 1st column is already filled
    
toronto_neigh_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant,Gastropub,Bar,Burger Joint,Bakery,Japanese Restaurant
1,Agincourt,Supermarket,Shopping Mall,Chinese Restaurant,Hong Kong Restaurant,Sushi Restaurant,Park,Vietnamese Restaurant,Bubble Tea Shop,Pool,Bakery
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Yoga Studio,Eastern European Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Park,Sandwich Place,Japanese Restaurant,Coffee Shop,Fried Chicken Joint,Liquor Store,Beer Store,Pizza Place,Fast Food Restaurant
4,"Alderwood, Long Branch",Convenience Store,Dance Studio,Gym,Pub,Sandwich Place,Fish & Chips Shop,Fish Market,Flea Market,Field,Eastern European Restaurant


## <strong><u> 4th Part: Clustering Neighbourhoods

In [57]:
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.03,0.0,0.01,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.030000,0.0,0.03,0.0,0.0,0.0,0.01,0.000000,0.0,0.0,0.0,0.0,0.02,0.0,0.01,0.02,0.0,0.0,0.000000,0.01,0.03

In [58]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)  #1 stands for axis
toronto_grouped_clustering.head()

,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.03,0.0,0.01,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.030000,0.0,0.03,0.0,0.0,0.0,0.01,0.000000,0.0,0.0,0.0,0.0,0.02,0.0,0.01,0.02,0.0,0.0,0.000000,0.01,0.03,0.01,0.0,0.0,0.0,0.0,0.0,0.06,0.0,0.0,0.

In [61]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)  #random state is for choosing random centers.
#fit can be either dataframe or array using np.asanyarray(df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 0, 1, 1, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 3, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 4, 1, 1, 2, 1, 1, 4, 1, 4, 1, 1, 1,
       1, 4, 1, 4, 1, 1, 4, 4, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 4, 1, 1, 1, 4, 1, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1,
       4, 1, 4, 1, 1, 1, 1, 1, 1, 1, 4], dtype=int32)

<i> Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [62]:
# add clustering labels
toronto_neigh_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)  #insert:Insert column into DataFrame at specified location.
#Parameters:	
#loc : int Insertion index--> Must verify 0 <= loc <= len(columns)
#column : string, number, or hashable object-->label of the inserted column
#value : int, Series, or array-like

In [63]:
#let's take a look at the first data.
toronto_neigh_venues_sorted.head(5)

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant,Gastropub,Bar,Burger Joint,Bakery,Japanese Restaurant
1,1,Agincourt,Supermarket,Shopping Mall,Chinese Restaurant,Hong Kong Restaurant,Sushi Restaurant,Park,Vietnamese Restaurant,Bubble Tea Shop,Pool,Bakery
2,0,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Yoga Studio,Eastern European Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
3,1,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Park,Sandwich Place,Japanese Restaurant,Coffee Shop,Fried Chicken Joint,Liquor Store,Beer Store,Pizza Place,Fast Food Restaurant
4,1,"Alderwood, Long Branch",Convenience Store,Dance Studio,Gym,Pub,Sandwich Place,Fish & Chips Shop,Fish Market,Flea Market,Field,Eastern European Restaurant


In [77]:
#Let's also see the data we got from wikipedia and their coordinates using geocoder
toronto_merged = df
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286
4,M7A,Queen's Park,Queen's Park,43.661150,-79.391715


In [78]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_neigh_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,4.0,Food & Drink Shop,Park,Bus Stop,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
1,M4A,North York,Victoria Village,43.730600,-79.313265,4.0,Park,Grocery Store,Yoga Studio,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Eastern European Restaurant
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,1.0,Coffee Shop,Bakery,Boat or Ferry,Theater,Health & Beauty Service,Mexican Restaurant,Spa,Brewery,Breakfast Spot,Pub
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286,1.0,Clothing Store,American Restaurant,Furniture / Home Store,Cosmetics Shop,Food Court,Toy / Game Store,Men's Store,Smoothie Shop,Coffee Shop,Bookstore
4,M7A,Queen's Park,Queen's Park,43.661150,-79.391715,1.0,Coffee Shop,Sandwich Place,Café,Pharmacy,Italian Restaurant,Park,Sushi Restaurant,Gastropub,College Cafeteria,College Auditorium


In [79]:
toronto_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,4.0,Food & Drink Shop,Park,Bus Stop,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
1,M4A,North York,Victoria Village,43.730600,-79.313265,4.0,Park,Grocery Store,Yoga Studio,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Eastern European Restaurant
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,1.0,Coffee Shop,Bakery,Boat or Ferry,Theater,Health & Beauty Service,Mexican Restaurant,Spa,Brewery,Breakfast Spot,Pub
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286,1.0,Clothing Store,American Restaurant,Furniture / Home Store,Cosmetics Shop,Food Court,Toy / Game Store,Men's Store,Smoothie Shop,Coffee Shop,Bookstore
4,M7A,Queen's Park,Queen's Park,43.661150,-79.391715,1.0,Coffee Shop,Sandwich Place,Café,Pharmacy,Italian Restaurant,Park,Sushi Restaurant,Gastropub,College Cafeteria,College Auditorium
5,M9A,Queen's Park,Queen's Park,43.662299,-79.528195,1.0,Coffee Shop,Sandwich Place,Café,Pharmacy,Italian Restaurant,Park,Sushi Restaurant,Gastropub,College Cafeteria,College Auditorium
6,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517,2.0,Home Service,Yoga Studio,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
7,M3B,North York,Don Mills North,43.749055,-79.362227,4.0,Burger Joint,Soccer Field,Park,Flower Shop,Flea Market,Food,Fish Market,Fish & Chips Shop,Eastern European Restaurant,Field
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.707535,-79.311773,1.0,Pizza Place,Fast Food Restaurant,Pharmacy,Gym / Fitness Center,Café,Breakfast Spot,Gastropub,Rock Climbing Spot,Bank,Intersection
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180,1.0,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Café,Diner,Bookstore,Hotel,Restaurant,Sporting Goods Shop


In [80]:
#As you see from the dataframe above the Foursquare API did not gave any data for the M9B,Etobicoke Neighborhood,thus we will drop that neighbourhood
toronto_merged.dropna(inplace=True,axis=0)
toronto_merged.reset_index(inplace=True)
toronto_merged

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M3A,North York,Parkwoods,43.752420,-79.329242,4.0,Food & Drink Shop,Park,Bus Stop,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
1,1,M4A,North York,Victoria Village,43.730600,-79.313265,4.0,Park,Grocery Store,Yoga Studio,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Eastern European Restaurant
2,2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,1.0,Coffee Shop,Bakery,Boat or Ferry,Theater,Health & Beauty Service,Mexican Restaurant,Spa,Brewery,Breakfast Spot,Pub
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286,1.0,Clothing Store,American Restaurant,Furniture / Home Store,Cosmetics Shop,Food Court,Toy / Game Store,Men's Store,Smoothie Shop,Coffee Shop,Bookstore
4,4,M7A,Queen's Park,Queen's Park,43.661150,-79.391715,1.0,Coffee Shop,Sandwich Place,Café,Pharmacy,Italian Restaurant,Park,Sushi Restaurant,Gastropub,College Cafeteria,College Auditorium
5,5,M9A,Queen's Park,Queen's Park,43.662299,-79.528195,1.0,Coffee Shop,Sandwich Place,Café,Pharmacy,Italian Restaurant,Park,Sushi Restaurant,Gastropub,College Cafeteria,College Auditorium
6,6,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517,2.0,Home Service,Yoga Studio,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
7,7,M3B,North York,Don Mills North,43.749055,-79.362227,4.0,Burger Joint,Soccer Field,Park,Flower Shop,Flea Market,Food,Fish Market,Fish & Chips Shop,Eastern European Restaurant,Field
8,8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.707535,-79.311773,1.0,Pizza Place,Fast Food Restaurant,Pharmacy,Gym / Fitness Center,Café,Breakfast Spot,Gastropub,Rock Climbing Spot,Bank,Intersection
9,9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180,1.0,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Café,Diner,Bookstore,Hotel,Restaurant,Sporting Goods Shop


<i>Finally, let's visualize the resulting clusters

In [82]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]  
#The RGB color model is an additive color model[1] in which red, green, and blue light are added together in various ways to reproduce a broad array of colors.
#rgb2hex: converts rgb to hex
#e,g:
#matplotlib.colors.to_hex([ 0.47, 0.0, 1.0 ])
#out:--->u'#7800ff'

# add markers to the map
markers_colors = []
for lat, lon,postcode, neigh, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'],toronto_merged['Postcode'],toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(postcode)+","+str(neigh) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1], #if cluster is 0 then the 1- will lead to a color of #ff0000 in the rainbow list!
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)  #fill_opacity fills the circle.if it is zero the circle is similar to a donut.
       
map_clusters

## <strong><u> 5th Part: Examine Clusters

#### Cluster 1

In [86]:
#toronto_merged.columns[[1]+[2]]
#out: Index(['Postcode', 'Borough'], dtype='object')

Index(['Postcode', 'Borough'], dtype='object')

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,M1V,-79.280187,0.0,Pharmacy,Yoga Studio,Eastern European Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant


#### Cluster 2

In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,-79.359166,1.0,Coffee Shop,Bakery,Boat or Ferry,Theater,Health & Beauty Service,Mexican Restaurant,Spa,Brewery,Breakfast Spot,Pub
3,M6A,-79.451286,1.0,Clothing Store,American Restaurant,Furniture / Home Store,Cosmetics Shop,Food Court,Toy / Game Store,Men's Store,Smoothie Shop,Coffee Shop,Bookstore
4,M7A,-79.391715,1.0,Coffee Shop,Sandwich Place,Café,Pharmacy,Italian Restaurant,Park,Sushi Restaurant,Gastropub,College Cafeteria,College Auditorium
5,M9A,-79.528195,1.0,Coffee Shop,Sandwich Place,Café,Pharmacy,Italian Restaurant,Park,Sushi Restaurant,Gastropub,College Cafeteria,College Auditorium
8,M4B,-79.311773,1.0,Pizza Place,Fast Food Restaurant,Pharmacy,Gym / Fitness Center,Café,Breakfast Spot,Gastropub,Rock Climbing Spot,Bank,Intersection
9,M5B,-79.378180,1.0,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Café,Diner,Bookstore,Hotel,Restaurant,Sporting Goods Shop
10,M6B,-79.448367,1.0,Pizza Place,Fast Food Restaurant,Grocery Store,Mediterranean Restaurant,Japanese Restaurant,Asian Restaurant,Latin American Restaurant,Sushi Restaurant,Yoga Studio,Event Space
11,M1C,-79.158725,1.0,Bar,History Museum,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
12,M3C,-79.343454,1.0,Bubble Tea Shop,Intersection,Beer Store,Coffee Shop,Grocery Store,Gym,Supermarket,Event Space,Falafel Restaurant,Farm
13,M4C,-79.306874,1.0,Bus Line,Grocery Store,Pizza Place,Pharmacy,Café,Market,Metro Station,Breakfast Spot,Middle Eastern Restaurant,Fast Food Restaurant


#### Cluster 3

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1B,-79.195517,2.0,Home Service,Yoga Studio,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
27,M3H,-79.448688,2.0,Home Service,Men's Store,Creperie,Cuban Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
48,M9L,-79.557028,2.0,Home Service,Rental Car Location,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Dumpling Restaurant
51,M3M,-79.496740,2.0,Home Service,Business Service,Yoga Studio,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fish & Chips Shop
60,M5N,-79.419120,2.0,Health & Beauty Service,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant


#### Cluster 4

In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,M1H,-79.239440,3.0,Playground,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
54,M6M,-79.484489,3.0,Construction & Landscaping,Coffee Shop,Fast Food Restaurant,Playground,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Eastern European Restaurant


#### Cluster 5

In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,-79.329242,4.0,Food & Drink Shop,Park,Bus Stop,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
1,M4A,-79.313265,4.0,Park,Grocery Store,Yoga Studio,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Eastern European Restaurant
7,M3B,-79.362227,4.0,Burger Joint,Soccer Field,Park,Flower Shop,Flea Market,Food,Fish Market,Fish & Chips Shop,Eastern European Restaurant,Field
17,M1E,-79.175193,4.0,Construction & Landscaping,Park,Gym / Fitness Center,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Eastern European Restaurant
20,M6E,-79.451018,4.0,Park,Sporting Goods Shop,Mexican Restaurant,Fast Food Restaurant,Beer Store,Bakery,Gym,Market,Women's Store,Department Store
34,M4J,-79.334175,4.0,Bar,Italian Restaurant,Farmers Market,Park,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Field
35,M5J,-79.362433,4.0,Harbor / Marina,Park,Pier,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
38,M2K,-79.380529,4.0,Construction & Landscaping,Park,Dog Run,Trail,Golf Driving Range,Dance Studio,Electronics Store,Food,Flower Shop,Flea Market
40,M4K,-79.355105,4.0,Park,Bus Line,Grocery Store,Discount Store,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Yoga Studio
47,M6L,-79.488301,4.0,Basketball Court,Bakery,Park,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Yoga Studio


In [ ]:
### Thank you for reading.

This notebook was created by [Georgios Trakakis](https://www.linkedin.com/in/georgios-trakakis/?originalSubdomain=uk) for the IBM Applied I hope you found thisclustering exercise in